In [ ]:
import cv2
import numpy as np
from shapely.geometry import Polygon
from ultralytics import YOLO
import os

video = r"C:\Users\Sajan Mevada\Downloads\20240116_161459 (2).mp4"
FIRST_FRAME_FOLDER = r"D:\projects\safety measurements\base\static\first_frame"
OUTPUT_FOLDER = r"D:\projects\safety measurements\base\static\output"
coordinates = [(41, 283), (237, 196), (602, 467), (112, 472)]


def count_persons_entered_restricted_area(video, coordinates):
    # Initialize YOLO model
    model = YOLO("yolov8n.pt")
    names = model.names

    # Open video for processing
    video_capture = cv2.VideoCapture(video)

    # Get video properties
    width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video_capture.get(cv2.CAP_PROP_FPS))

    # Define the codec and create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # MP4V codec
    output_video_path = os.path.join(OUTPUT_FOLDER, 'output_video.mp4')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (720, 480))

    # Create a Shapely Polygon from the user-input coordinates
    restricted_area_shapely = Polygon(coordinates)

    # Define colors
    color_restricted_entered = (255, 0, 0)  # Blue
    color_restricted_empty = (255, 255, 255)  # White
    color_person_inside = (0, 0, 255)  # Red
    color_person_outside = (0, 255, 0)  # Green

    persons_entered_count = 0  # Track the count of persons entered

    while True:
        ret, frame = video_capture.read()

        if not ret:
            break

        # Resize the frame
        frame = cv2.resize(frame, (720, 480))

        # YOLO processing
        results = model(frame, classes=[0])
        boxes = results[0].boxes

        # Draw restricted area
        cv2.polylines(frame, [np.array(coordinates)], isClosed=True, color=color_restricted_empty, thickness=2)

        for box in boxes:
            class_id = int(box.cpu().cls[0])
            confidence = float(box.cpu().conf)
            x1, y1, x2, y2 = box.cpu().xyxy[0]

            x3, y3 = x1 + abs(x2 - x1), y1
            x4, y4 = x1, y1 + abs(y1 - y2)

            person_polygon_shapely = Polygon([(x1, y1), (x4, y4), (x2, y2), (x3, y3)])
            intersection_area = restricted_area_shapely.intersection(person_polygon_shapely).area
            union_area = restricted_area_shapely.union(person_polygon_shapely).area
            iou = intersection_area / union_area if union_area > 0 else 0

            # Check if person is inside or outside the restricted area
            if names.get(class_id) == 'person':
                if iou > 0.01:
                    persons_entered_count += 1
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color_person_inside, 2)
                    cv2.putText(frame, f'Confidence: {confidence:.2f}', (int(x1), int(y1) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                    # Draw restricted area in blue when a person is inside
                    cv2.polylines(frame, [np.array(coordinates)], isClosed=True, color=color_restricted_entered,
                                  thickness=2)
                else:
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color_person_outside, 2)

        # Display count of persons entered in the top-left corner
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(frame, f'Persons Entered: {persons_entered_count}', (10, 30), font, 0.8, (0, 255, 255), 2)

        # Write the frame to the output video
        out.write(frame)

        # Display the processed frame
        cv2.imshow('Processed Video', frame)

        if cv2.waitKey(1) == ord("q"):
            break

    # Release the video capture object
    video_capture.release()

    # Release the VideoWriter
    out.release()

    # Destroy all OpenCV windows
    cv2.destroyAllWindows()

    return persons_entered_count